In [27]:

# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np

In [28]:
data = r"C:\Users\agaro\Documents\GitHub\Chiron\algorithm\dataset1.csv"

df = pd.read_csv(data)
print(df.head())

                                                link  \
0  https://www.infowars.com/posts/the-wall-is-rea...   
1  https://www.infowars.com/posts/post-vaccinatio...   
2  https://www.infowars.com/posts/half-of-all-wom...   
3  https://www.infowars.com/posts/high-levels-of-...   
4  https://www.infowars.com/posts/epidemiologist-...   

                                               title  annotation  \
0  The Wall Is Real: Half of All Thirty-Something...           0   
1  “Post-Vaccination Syndrome:” New Paper Identif...           0   
2  Half of All Women in US Aged 30-35 Experiencin...           0   
3  High Levels of Microplastics Found in Lungs of...           0   
4  Epidemiologist Reveals New Data Linking Covid ...           0   

                                             content  
0  It’s a staple of the so-called “manosphere”—th...  
1  A new condition called “post-vaccination syndr...  
2  Half of all women in their 30s are now reporti...  
3  The lungs of birds contain sign

In [29]:
import string
string.punctuation

# Checks the contents if there are empty or missing values

uncleanContents = df[df['content'].isna() | (df['content'].str.strip() == '')].index

# Print the missing/empty indices
print(f'Indices with missing or empty content: {uncleanContents}')
# print('empty content example: ' + (df['content'])[217])
# Fill NaN values with empty strings
df['content'].fillna('', inplace=True)

print(f"First file rows: {len(df)}")

Indices with missing or empty content: Index([58, 185, 203, 210, 274, 289, 2018, 2019, 2020, 2021], dtype='int64')
First file rows: 2801


First step we are doing is data cleaning and preprocessing

In [30]:
#defining the function to remove punctuation
def remove_punctuation(text):
    punctuationfree = "".join([i for i in text if i not in string.punctuation and i not in ["'", '"', '—', '“', '”', '’', '––', '–']])
    return punctuationfree
#storing the puntuation free text
df['clean_msg']= df['content'].apply(lambda x:remove_punctuation(x))
df.head()

,link,title,annotation,content,clean_msg
0,https://www.infowars.com/posts/the-wall-is-rea...,The Wall Is Real: Half of All Thirty-Something...,0,It’s a staple of the so-called “manosphere”—th...,Its a staple of the socalled manospherethat gl...
1,https://www.infowars.com/posts/post-vaccinatio...,“Post-Vaccination Syndrome:” New Paper Identif...,0,A new condition called “post-vaccination syndr...,A new condition called postvaccination syndrom...
2,https://www.infowars.com/posts/half-of-all-wom...,Half of All Women in US Aged 30-35 Experiencin...,0,Half of all women in their 30s are now reporti...,Half of all women in their 30s are now reporti...
3,https://www.infowars.com/posts/high-levels-of-...,High Levels of Microplastics Found in Lungs of...,0,The lungs of birds contain significant quantit...,The lungs of birds contain significant quantit...
4,https://www.infowars.com/posts/epidemiologist-...,Epidemiologist Reveals New Data Linking Covid ...,0,"Nicolas Hulscher, an epidemiologist with the M...",Nicolas Hulscher an epidemiologist with the Mc...


In [31]:
# standardization: making all letters into lowercase
df['msg_lower']= df['clean_msg'].apply(lambda x: x.lower())
df.head()

,link,title,annotation,content,clean_msg,msg_lower
0,https://www.infowars.com/posts/the-wall-is-rea...,The Wall Is Real: Half of All Thirty-Something...,0,It’s a staple of the so-called “manosphere”—th...,Its a staple of the socalled manospherethat gl...,its a staple of the socalled manospherethat gl...
1,https://www.infowars.com/posts/post-vaccinatio...,“Post-Vaccination Syndrome:” New Paper Identif...,0,A new condition called “post-vaccination syndr...,A new condition called postvaccination syndrom...,a new condition called postvaccination syndrom...
2,https://www.infowars.com/posts/half-of-all-wom...,Half of All Women in US Aged 30-35 Experiencin...,0,Half of all women in their 30s are now reporti...,Half of all women in their 30s are now reporti...,half of all women in their 30s are now reporti...
3,https://www.infowars.com/posts/high-levels-of-...,High Levels of Microplastics Found in Lungs of...,0,The lungs of birds contain significant quantit...,The lungs of birds contain significant quantit...,the lungs of birds contain significant quantit...
4,https://www.infowars.com/posts/epidemiologist-...,Epidemiologist Reveals New Data Linking Covid ...,0,"Nicolas Hulscher, an epidemiologist with the M...",Nicolas Hulscher an epidemiologist with the Mc...,nicolas hulscher an epidemiologist with the mc...


In [32]:
import re
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading punkt_tab: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [33]:
#defining function for tokenization

#applying function to the column
df['msg_tokenized'] = df['msg_lower'].apply(lambda x: word_tokenize(x))

print(df['msg_tokenized'][1])

['a', 'new', 'condition', 'called', 'postvaccination', 'syndrome', 'has', 'been', 'acknowledged', 'by', 'yale', 'scientists', 'investigating', 'the', 'longterm', 'aftereffects', 'of', 'covid19', 'vaccination', 'the', 'condition', 'is', 'characterised', 'by', 'a', 'variety', 'of', 'troubling', 'symptoms', 'including', 'brain', 'fog', 'insomnia', 'tinnitus', 'and', 'heart', 'palpitations', 'and', 'resembles', 'the', 'condition', 'dubbed', 'long', 'covid', 'the', 'symptoms', 'typically', 'develop', 'a', 'day', 'or', 'two', 'after', 'vaccination', 'and', 'can', 'persist', 'for', 'months', 'or', 'even', 'years', 'worsening', 'over', 'time', 'its', 'clear', 'that', 'some', 'individuals', 'are', 'experiencing', 'significant', 'challenges', 'after', 'vaccination', 'our', 'responsibility', 'as', 'scientists', 'and', 'clinicians', 'is', 'to', 'listen', 'to', 'their', 'experiences', 'rigorously', 'investigate', 'the', 'underlying', 'causes', 'and', 'seek', 'ways', 'to', 'help', 'said', 'cosenior'

In [34]:
#importing nlp library
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# stop_words = set(stopwords.words('english'))
#Stop words present in the library
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [35]:
import nltk
# Download stopwords if you haven't already
nltk.download('stopwords')

# Initialize stopwords (use only ONE of these lines)
stopwords = set(nltk.corpus.stopwords.words('english'))

# Add custom stopwords
custom_stopwords = ["'", '"', '—', '“', '”', '’', '––', '–', 'said', 'human', 'people', 'health', 'also', 'would', 'could', 'said', '2025', '2024', 'dec', 'feb', 'jan', 'nov', 'oct', 'sept', 'aug', 'july', 'june',
                                                                                                'may', 'april', 'march']
stopwords = stopwords | set(custom_stopwords)

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [36]:
# defining functions for removing stopwords
def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output

df['no_stopwords']= df['msg_tokenized'].apply(lambda x:remove_stopwords(x))
print(df['no_stopwords'][1])

['new', 'condition', 'called', 'postvaccination', 'syndrome', 'acknowledged', 'yale', 'scientists', 'investigating', 'longterm', 'aftereffects', 'covid19', 'vaccination', 'condition', 'characterised', 'variety', 'troubling', 'symptoms', 'including', 'brain', 'fog', 'insomnia', 'tinnitus', 'heart', 'palpitations', 'resembles', 'condition', 'dubbed', 'long', 'covid', 'symptoms', 'typically', 'develop', 'day', 'two', 'vaccination', 'persist', 'months', 'even', 'years', 'worsening', 'time', 'clear', 'individuals', 'experiencing', 'significant', 'challenges', 'vaccination', 'responsibility', 'scientists', 'clinicians', 'listen', 'experiences', 'rigorously', 'investigate', 'underlying', 'causes', 'seek', 'ways', 'help', 'cosenior', 'author', 'harlan', 'krumholz', 'researchers', 'looked', 'blood', 'samples', '42', 'patients', 'experienced', 'symptoms', 'postvaccination', 'syndrome', '22', 'syndrome', 'found', 'lower', 'levels', 'true', 'crucial', 'white', 'bloodcell', 'types', 'lower', 'level

In [37]:
from nltk.stem.porter import PorterStemmer
#defining the object for stemming
porter_stemmer = PorterStemmer()

In [38]:
#defining a function for stemming
def stemming(text):
  stem_text = [porter_stemmer.stem(word) for word in text]
  return stem_text

# consideration: the results of stemming here is not human readable, but according to chatgpt: However, if your goal is purely text classification (like Naive Bayes or TF-IDF), these changes may not be a problem, as long as your model learns meaningful patterns.
df['msg_stemmed'] = df['no_stopwords'].apply(lambda x: stemming(x))
print(df['msg_stemmed'][1])

['new', 'condit', 'call', 'postvaccin', 'syndrom', 'acknowledg', 'yale', 'scientist', 'investig', 'longterm', 'aftereffect', 'covid19', 'vaccin', 'condit', 'characteris', 'varieti', 'troubl', 'symptom', 'includ', 'brain', 'fog', 'insomnia', 'tinnitu', 'heart', 'palpit', 'resembl', 'condit', 'dub', 'long', 'covid', 'symptom', 'typic', 'develop', 'day', 'two', 'vaccin', 'persist', 'month', 'even', 'year', 'worsen', 'time', 'clear', 'individu', 'experienc', 'signific', 'challeng', 'vaccin', 'respons', 'scientist', 'clinician', 'listen', 'experi', 'rigor', 'investig', 'underli', 'caus', 'seek', 'way', 'help', 'cosenior', 'author', 'harlan', 'krumholz', 'research', 'look', 'blood', 'sampl', '42', 'patient', 'experienc', 'symptom', 'postvaccin', 'syndrom', '22', 'syndrom', 'found', 'lower', 'level', 'true', 'crucial', 'white', 'bloodcel', 'type', 'lower', 'level', 'antibodi', 'covid19', 'spike', 'protein', 'case', 'higher', 'level', 'spike', 'protein', 'dont', 'know', 'level', 'spike', 'prot

In [39]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
#defining the object for Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()

[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [40]:
#defining the function for lemmatization
def lemmatizer(text):
  lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
  return lemm_text

df['msg_lemmatized'] = df['no_stopwords'].apply(lambda x:lemmatizer(x))
print(df['msg_lemmatized'][1])

['new', 'condition', 'called', 'postvaccination', 'syndrome', 'acknowledged', 'yale', 'scientist', 'investigating', 'longterm', 'aftereffect', 'covid19', 'vaccination', 'condition', 'characterised', 'variety', 'troubling', 'symptom', 'including', 'brain', 'fog', 'insomnia', 'tinnitus', 'heart', 'palpitation', 'resembles', 'condition', 'dubbed', 'long', 'covid', 'symptom', 'typically', 'develop', 'day', 'two', 'vaccination', 'persist', 'month', 'even', 'year', 'worsening', 'time', 'clear', 'individual', 'experiencing', 'significant', 'challenge', 'vaccination', 'responsibility', 'scientist', 'clinician', 'listen', 'experience', 'rigorously', 'investigate', 'underlying', 'cause', 'seek', 'way', 'help', 'cosenior', 'author', 'harlan', 'krumholz', 'researcher', 'looked', 'blood', 'sample', '42', 'patient', 'experienced', 'symptom', 'postvaccination', 'syndrome', '22', 'syndrome', 'found', 'lower', 'level', 'true', 'crucial', 'white', 'bloodcell', 'type', 'lower', 'level', 'antibody', 'covi

Second step we will have is processing

In [41]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB

# Assuming df contains 'content' and 'annotation' columns
def prepare_data(df):
    # Process content column
    X = df['msg_lemmatized'].apply(lambda tokens: " ".join(tokens) if isinstance(tokens, list) else tokens)
    y = df['annotation']

    print(X[1])

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Vectorize text data
    vectorizer = CountVectorizer()
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)

    return X_train_vec, X_test_vec, y_train, y_test, vectorizer

In [42]:


def train_naive_bayes(X_train_vec, y_train):
    # Initialize and train Gaussian Naive Bayes classifier
    nb_model = MultinomialNB()
    nb_model.fit(X_train_vec, y_train)  # No need for .toarray() here
    return nb_model

def train_svm(X_train_vec, y_train):
    # Initialize and train SVM classifier with increased max_iter
    svm_model = LinearSVC(random_state=42, max_iter=5000)  # Increase max_iter for convergence
    svm_model.fit(X_train_vec, y_train)
    return svm_model

def train_logistic_regression(X_train_vec, y_train):
    # Train Logistic Regression with a specified solver
    logistic_model = LogisticRegression(solver='lbfgs', max_iter=1000, random_state=42)  # Use 'lbfgs' solver
    logistic_model.fit(X_train_vec, y_train)
    return logistic_model

def evaluate_model(model, X_test_vec, y_test, model_name):
    # Make predictions
    y_pred = model.predict(X_test_vec)

    # Calculate and print metrics
    accuracy = accuracy_score(y_test, y_pred)
    print(f'\n{model_name} Results:')
    print(f'Accuracy: {accuracy * 100:.2f}%')
    print('\nClassification Report:')
    print(classification_report(y_test, y_pred))

    return y_pred


In [43]:
# Main execution
from sklearn.model_selection import cross_val_score

# Main execution with cross-validation and validation split
def main(df):
    # Prepare data
    X_train_vec, X_test_vec, y_train, y_test, vectorizer = prepare_data(df)

   # Cross-validation for Naive Bayes
    nb_model = train_naive_bayes(X_train_vec, y_train)
    nb_cv_scores = cross_val_score(nb_model, X_train_vec, y_train, cv=5, scoring='accuracy')
    print(f'Naive Bayes Cross-Validation Accuracy: {nb_cv_scores.mean() * 100:.2f}%')

    # Cross-validation for SVM
    svm_model = train_svm(X_train_vec, y_train)
    svm_cv_scores = cross_val_score(svm_model, X_train_vec, y_train, cv=5, scoring='accuracy')
    print(f'SVM Cross-Validation Accuracy: {svm_cv_scores.mean() * 100:.2f}%')

    # Logistic regression
    logistic_model = train_logistic_regression(X_train_vec, y_train)
    logistic_regression_cv_scores = cross_val_score(logistic_model, X_train_vec, y_train, cv=5, scoring='accuracy')
    print(f'Logistic Regression Cross-Validation Accuracy: {logistic_regression_cv_scores.mean() * 100:.2f}%')

    # Train and evaluate Naive Bayes
    nb_predictions = evaluate_model(nb_model, X_test_vec, y_test, "Naive Bayes")

    # Train and evaluate SVM
    svm_predictions = evaluate_model(svm_model, X_test_vec, y_test, "Support Vector Machine")

    # Train and evaluate logistic regression
    logistic_regression_predictions = evaluate_model(logistic_model, X_test_vec, y_test, "Logistic Regression")


    return {
        'naive_bayes': {
            'model': nb_model,
            'predictions': nb_predictions,
            'cv_scores': nb_cv_scores
        },
        'svm': {
            'model': svm_model,
            'predictions': svm_predictions,
            'cv_scores': svm_cv_scores
        },
        'logistic_regression': {
            'model': logistic_model,
            'predictions': logistic_regression_predictions,
            'val_accuracy': logistic_regression_cv_scores
        },
        'vectorizer': vectorizer
    }

In [44]:
# Example usage
if __name__ == "__main__":
    # Assuming df is your DataFrame
    results = main(df)

new condition called postvaccination syndrome acknowledged yale scientist investigating longterm aftereffect covid19 vaccination condition characterised variety troubling symptom including brain fog insomnia tinnitus heart palpitation resembles condition dubbed long covid symptom typically develop day two vaccination persist month even year worsening time clear individual experiencing significant challenge vaccination responsibility scientist clinician listen experience rigorously investigate underlying cause seek way help cosenior author harlan krumholz researcher looked blood sample 42 patient experienced symptom postvaccination syndrome 22 syndrome found lower level true crucial white bloodcell type lower level antibody covid19 spike protein case higher level spike protein dont know level spike protein causing chronic symptom participant pvs postvaccination syndrome didnt measurable spike proteinbut one mechanism underlying syndrome cosenior study author akiko iwasaki researcher bel